# Data reading

In [6]:
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2024-06-12 13:37:39--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt’

input.txt           100%[===================>]   1.06M  --.-KB/s    in 0.06s   

2024-06-12 13:37:40 (16.9 MB/s) - ‘input.txt’ saved [1115394/1115394]



In [7]:
with open('input.txt', 'r', encoding='utf-8') as f:
  text = f.read()

In [8]:
print(len(text), len(text.split()), len(set(text)))

1115394 202651 65


# Data processing

In [9]:
chars = sorted(set(text))
vocab_size = len(chars)
print(f"{vocab_size=}\n<S>{'|'.join(chars)}<E>")

vocab_size=65
<S>
| |!|$|&|'|,|-|.|3|:|;|?|A|B|C|D|E|F|G|H|I|J|K|L|M|N|O|P|Q|R|S|T|U|V|W|X|Y|Z|a|b|c|d|e|f|g|h|i|j|k|l|m|n|o|p|q|r|s|t|u|v|w|x|y|z<E>


In [10]:
# creating vocabulary and mapping and reverse mapping
itos = {i:c for i, c in enumerate(chars)}
stoi = {c:i for i, c in itos.items()}
encode = lambda s : [stoi[c] for c in s]
decode = lambda e : ''.join([itos[i] for i in e])

sample_text = "what's up"
print(encode(sample_text))
print(decode(encode(sample_text)))

[61, 46, 39, 58, 5, 57, 1, 59, 54]
what's up


## few notes about tokenizer usage

Try other encodings as well.
e.g.
* SentencePiece
* Also BPE's fast implementation is tiktoken used in GPT models created by OpenAI
* OpenAI's tokenizer demo - https://platform.openai.com/tokenizer
* Third party tokenizer demo - https://tiktokenizer.vercel.app/
* tiktoken repo - https://github.com/openai/tiktoken
* sample tiktoken usage cookbook - https://github.com/openai/openai-cookbook/blob/main/examples/How_to_count_tokens_with_tiktoken.ipynb


---


**ToDo** - Learn about BPE tokenizer process in depth.

In [12]:
#!pip install tiktoken
import tiktoken
enc = tiktoken.get_encoding('gpt2')
print(enc.n_vocab)
print(enc.encode(sample_text))
print(enc.decode(enc.encode(sample_text)))


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 5.8 MB/s eta 0:00:00
50257
[10919, 338, 510]
what's up


## data processing continued (with character encoding)

In [13]:
import torch
#encode text into token
data = torch.tensor(encode(text), dtype=torch.long)
print(f"{data.shape=}, {data.dtype=}")
print(data[:100])

data.shape=torch.Size([1115394]), data.dtype=torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59])


In [41]:
# train, val split
split = int(0.9*len(data))
train_data = data[:split]
val_data = data[split:]
print(f"{train_data.shape=}, {val_data.shape=}")

train_data.shape=torch.Size([1003854]), val_data.shape=torch.Size([111540])


In [10]:
# get batches of data in block_size

block_size = 8 # also called as context length
batch_size = 4 # for parallel processing

torch.manual_seed(1337)
def get_batch(split):
  data = train_data if split=='train' else val_data
  batch_ixs = torch.randint(0, len(data) - block_size, (batch_size,))
  x = torch.stack([data[ix : ix + block_size] for ix in batch_ixs])
  y = torch.stack([data[ix + 1: ix + 1 + block_size] for ix in batch_ixs])
  '''
  if x indices are  [3,4,5,6,7,8,9, 10]
  y indices will be [4,5,6,7,8,9,10,11]

  torch.stack will just stack in one extra dimension in front
  '''
  return x, y

xb, yb = get_batch('train')
print(f"{xb.shape=}, {yb.shape=}")
print(xb)
print(yb)

for bt in range(batch_size):
  for bl in range(block_size):
    context = xb[bt, :bl+1]
    output  = yb[bt, bl]
    print(f"{decode(context.numpy())} --> {decode([output.item()])}")


xb.shape=torch.Size([4, 8]), yb.shape=torch.Size([4, 8])
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])
L --> e
Le --> t
Let --> '
Let' --> s
Let's -->  
Let's  --> h
Let's h --> e
Let's he --> a
f --> o
fo --> r
for -->  
for  --> t
for t --> h
for th --> a
for tha --> t
for that -->  
n --> t
nt -->  
nt  --> t
nt t --> h
nt th --> a
nt tha --> t
nt that -->  
nt that  --> h
M --> E
ME --> O
MEO --> :
MEO: --> 

MEO:
 --> I
MEO:
I -->  
MEO:
I  --> p
MEO:
I p --> a


# lets start with simple bigram model

learn more about torch libraries - specifically start with nn.Module

In [11]:
xb.shape, yb.shape

(torch.Size([4, 8]), torch.Size([4, 8]))

In [2]:
import torch.nn as nn
import torch.nn.functional as F

class BigramLanguageModel(nn.Module):
  def __init__(self, vocab_size):
    super().__init__()
    self.token_embedding = nn.Embedding(vocab_size, vocab_size)

  def forward(self, x, targets=None):
    logits = self.token_embedding(x) # converts shape(B, T) to shape(B, T, C) | B-batch, T-Time, C-Channel(Embedding_size)

    loss = None
    if targets is not None:
      B,T,C = logits.shape
      # read documentation of cross_entropy for reason of this reshaping -
      # https://pytorch.org/docs/stable/generated/torch.nn.functional.cross_entropy.html#torch.nn.functional.cross_entropy
      logits = logits.view(B*T, C)
      targets = targets.view(B*T) # or just -1
      loss = F.cross_entropy(logits, targets)

    return logits, loss

  def generate(self, idx, max_new_tokens):
    for _ in range(max_new_tokens):
      logits, _ = self(idx)             # forward pass to get logits, loss is not needed
      logits = logits[:, -1, :]         # logits will be of shape B,T,C but we only care about last character i.e. last T
      probs = F.softmax(logits, dim=1)  # prob using softmax along the channel dimension
      next_idx = torch.multinomial(probs, 1)
      # that is, we are training with (4,8) -> (4,8) shape. But feeding(1, +inf) shape to get (1, +inf) out and then just consume only last T.
      # validate this fact
      #print(f"debug | {decode(idx[0].numpy())} --> {decode([next_idx[0].item()])}") # 0-index to take from first batch
      idx = torch.cat([idx, next_idx], dim=1)
    return idx

In [28]:
model = BigramLanguageModel(vocab_size)
logits, loss = model(xb, yb)
print(f"{logits=}, {loss=}")

logits=tensor([[ 0.0926,  0.5827, -0.6639,  ...,  0.8576,  0.5767, -0.1432],
        [-0.0906, -1.1605,  1.2265,  ..., -0.3734, -0.5206,  0.7613],
        [-0.8223, -0.5409,  0.1870,  ..., -1.3379,  0.0693,  2.2249],
        ...,
        [-0.2609,  0.7606, -0.9424,  ..., -1.5740,  0.8685, -0.8768],
        [ 0.6698,  0.2120,  0.5794,  ...,  0.8288,  2.3338,  0.9831],
        [-0.7894, -1.1862,  0.3646,  ..., -1.4089,  1.4106, -0.2585]],
       grad_fn=<ViewBackward0>), loss=tensor(4.6343, grad_fn=<NllLossBackward0>)


In [37]:
# we are starting with a batch and 1 context length to start with

def generate_with_no_context(max_new_tokens=20):
  #model.generate(torch.tensor([stoi[' ']]*block_size).view(1, -1), max_new_tokens=10)
  inp = torch.zeros((1,1), dtype=torch.long) # zero is newline char
  out = model.generate(inp, max_new_tokens=max_new_tokens)
  print(decode(out[0].tolist())) # 0-index to pick batch index

generate_with_no_context()

## lets try to create and use an optimizer to train model
* read API and docs here - https://pytorch.org/docs/stable/optim.html
* **ToDo** - learn about different optimizers, esp Adam(W)

In [33]:
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3)

In [34]:
batch_size = 32
max_steps = 100000
for i in range(max_steps):
  xb, yb = get_batch('train')
  logits, loss = model(xb, yb)
  optimizer.zero_grad(set_to_none=True) # this set_to_none is for performance optimization. but can lead to error if try to access grad
  loss.backward()
  optimizer.step()
  if i%10000==0:
    print(loss.item())

4.727841854095459
2.5576331615448
2.4996368885040283
2.397784471511841
2.4928104877471924
2.463300943374634
2.507370948791504
2.4625484943389893
2.5009632110595703
2.4473514556884766


In [39]:
generate_with_no_context(max_new_tokens=200)



Wham'de l:
MNUTLAs?
PA k My the thofim,
ARKis, her stoofous w weryon IO:
apr ce f ar sor n!
So,
Towinccan, hen.
bu s st LYCLOry aithot: twoforoove nd IO:
Youl wngd ha th a ou
Co heishimongor we poono


# matrix multiplcation tricks

* try examples by hand in notebook

In [77]:
# find avg of Cs across all previous Ts(including itself) for each batch(B) separately
# hint try pre mat multiple by lower traingle matrix of ones
# but if u avg out this prematrix. u can get avg also.
# but for (B,T,C) we need pre-mul matrix of size (B,T, T )

# |1 0 0|     |A|     |A    |
# |1 1 0|  @  |B|  =  |A+B  |
# |1 1 1|     |C|     |A+B+C|

In [102]:
torch.manual_seed(42)
B, T, C = 2,3,4
x = torch.randint(0, 10, (B,T,C)).float()
x

tensor([[[2., 7., 6., 4.],
         [6., 5., 0., 4.],
         [0., 3., 8., 4.]],

        [[0., 4., 1., 2.],
         [5., 5., 7., 6.],
         [9., 6., 3., 1.]]])

In [84]:
wei = torch.tril(torch.ones((B,T,T)))
# because of how torch only cares about merging dimensions - last of first and first of last. So B is not needed, it will just be broadcasted
wei = torch.tril(torch.ones((T,T)))
print(wei)
wei = wei / torch.sum(wei, dim=-1, keepdims=True)
print(wei)

tensor([[1., 0., 0.],
        [1., 1., 0.],
        [1., 1., 1.]])
tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]])


In [83]:
wei @ x

tensor([[[2.0000, 7.0000, 6.0000, 4.0000],
         [4.0000, 6.0000, 3.0000, 4.0000],
         [2.6667, 5.0000, 4.6667, 4.0000]],

        [[0.0000, 4.0000, 1.0000, 2.0000],
         [2.5000, 4.5000, 4.0000, 4.0000],
         [4.6667, 5.0000, 3.6667, 3.0000]]])

In [86]:
# in summary
wei = torch.tril(torch.ones((T,T)))
wei = wei / torch.sum(wei, dim=-1, keepdims=True)
way1 = wei @ x
way1

tensor([[[2.0000, 7.0000, 6.0000, 4.0000],
         [4.0000, 6.0000, 3.0000, 4.0000],
         [2.6667, 5.0000, 4.6667, 4.0000]],

        [[0.0000, 4.0000, 1.0000, 2.0000],
         [2.5000, 4.5000, 4.0000, 4.0000],
         [4.6667, 5.0000, 3.6667, 3.0000]]])

* another intersting way
* try to walk through manually urself - why and how it works.

In [100]:
# another way using softmax on -inf
tril = torch.tril(torch.ones((T,T)))
wei = torch.zeros((T,T)) # this will be used in self-attention, but this wei wont be just zero but the interaction between k & q
wei = wei.masked_fill(tril==0, value=float('-inf'))
wei = torch.softmax(wei, dim=-1)
print(wei)
way2 = wei @ x
print(way2)
torch.equal(way1, way2)

tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]])
tensor([[[2.0000, 7.0000, 6.0000, 4.0000],
         [4.0000, 6.0000, 3.0000, 4.0000],
         [2.6667, 5.0000, 4.6667, 4.0000]],

        [[0.0000, 4.0000, 1.0000, 2.0000],
         [2.5000, 4.5000, 4.0000, 4.0000],
         [4.6667, 5.0000, 3.6667, 3.0000]]])


True

# let's head on to self-attention

In [15]:
torch.manual_seed(1337)
B, T, C = 4,8,32
x = torch.randn((B,T,C)).float()

In [21]:
# single head attention
head_size = 16
key = nn.Linear(C, head_size, bias=False)
query = nn.Linear(C, head_size, bias=False)

k = key(x)    # (B,T,head_size)
q = query(x)  # (B,T,head_size)

wei = q @ k.transpose(-2, -1) * head_size**-0.5# (B,T,h)@(B,h,T) --> (B,T,T) | now, this `wei` is interaction matrix not just zero or uniform
# scaling is to control variance of `wei` before feeding to softmax. Otherwise if this scaling is not done, weights will be very high and softmax will cnvert focus to very small number of other tokens
# now, we can do the same ops - masking, softmax & multiplication (but to v similar to  k & q)

tril = torch.tril(torch.ones((T,T)))
wei = wei.masked_fill(tril==0, value=float('-inf')) # this masking is what makes this as decoder. Otherwise, encoder can look upto all chars in given input
                                                    # also, difference in decoder is it has another sublayer which takers encoder's input in 2 of k,q,v (which ones?). Also, only in first layer or all layers? ToDo - figure out
wei = torch.softmax(wei, dim=-1)

value = nn.Linear(C, head_size, bias=False)
v = value(x)

out = wei @ v # (B,T,T) @ (B,T,h)
print(out.shape) # (B, T, h) --> one head output

torch.Size([4, 8, 16])


In [22]:
wei[0]

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2853, 0.7147, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2858, 0.3704, 0.3437, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2679, 0.3740, 0.2292, 0.1289, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.1926, 0.1797, 0.1312, 0.1444, 0.3521, 0.0000, 0.0000, 0.0000],
        [0.1533, 0.1322, 0.2156, 0.2225, 0.0987, 0.1777, 0.0000, 0.0000],
        [0.0863, 0.1575, 0.1328, 0.1596, 0.1788, 0.1199, 0.1652, 0.0000],
        [0.1044, 0.1764, 0.1101, 0.0950, 0.1406, 0.1058, 0.1436, 0.1241]],
       grad_fn=<SelectBackward0>)